### Transformer 学习

Transformer 是文章《Attention Is All You Need》提出的一个模型结构，是 NLP 领域非常有创新性的成果。基于 Transformer 构建的模型 Bert，更是在多个自然语言任务上都取得了革命性的效果，可见 Transformer 的厉害之处。

Transformer 的创新之处是完全摒弃了循环和卷积的思路，只用到了注意力机制。这一架构的主要优点是层输出可以并行计算，而非像 RNN 这样的序列计算，而且能学习长距离的依赖。

先看看 Transformer 的整体架构：
<img src="images/model_architecture.png" width="700">
下面，一步步地剖析这个架构。

#### 1. Positional encoding (位置编码)

由于 Transformer 不包括循环或卷积，所以为了标记单词在句子中的相对位置，模型使用了 positional encoding。positional encoding 被加到 word embedding 向量中作为最终的输入。加入位置编码后，每个词根据它们的语义相似度和在句子中的位置来决定它们的最终相似度。

文章使用的位置编码为：
$$PE(pos,2i)=sin(pos/10000^{2i/d_{model}})\\
PE(pos,2i+1)=cos(pos/10000^{2i/d_{model}})$$

其中，$pos$ 表示位置，$i$ 表示维度。文章也尝试用学习好的位置编码，发现这两种方法效果差不多。